In [2]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import glob
import os
import pandas as pd

import efficientnet.tfkeras as efn

In [3]:
train_image_path = glob.glob('train\\*\\*.jpg')
labels = [int(i.split('\\')[1])-1 for i in train_image_path]
labels

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [4]:
# Step1 使用 tf.data.Dataset.from_tensor_slices 进行加载
image_ds = tf.data.Dataset.from_tensor_slices((train_image_path,labels))

In [5]:
val_count = int(len(labels)*0.2)
train_count = len(labels)-val_count

In [6]:
def load_preprocess_image(path,label):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image,channels=3)
    image = tf.image.resize(image,[300,300])
    image = tf.image.random_crop(image,[260,260,3])
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    image = tf.image.random_brightness(image,0.2)
    
    image = tf.cast(image,tf.float32)
    image = image/255
    label = tf.reshape(label,[1])
    return image,label

In [7]:
def load_preprocess_image_test(path,label):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image,channels=3)
    image = tf.image.resize(image,[260,260])
    image = tf.cast(image,tf.float32)
    image = image/255
    label = tf.reshape(label,[1])
    return image,label

In [8]:
# 选取验证集
image_train_ds = image_ds.skip(val_count)
image_val_ds = image_ds.take(val_count)

# Step3 预处理 (预处理函数在下面)
AUTOTUNE = tf.data.experimental.AUTOTUNE
image_train_ds = image_train_ds.map(load_preprocess_image,num_parallel_calls=AUTOTUNE)
image_val_ds = image_val_ds.map(load_preprocess_image_test,num_parallel_calls=AUTOTUNE)

In [9]:
BATCH_SIZE = 32
# shuffle 打乱数据 batch设置 batch size repeat设置迭代次数(迭代2次) test数据集不需要
image_train_ds = image_train_ds.repeat().shuffle(train_count).batch(BATCH_SIZE)
image_train_ds = image_train_ds.prefetch(AUTOTUNE)#预取,GPU，CPU加速

image_val_ds = image_val_ds.batch(BATCH_SIZE)
image_val_ds = image_val_ds.prefetch(AUTOTUNE)#预取

In [10]:
covn_base = efn.EfficientNetB5(weights='imagenet',
                               input_shape=(260,260,3),
                               include_top=False,
                               pooling='avg')

In [11]:
model = keras.Sequential()
model.add(covn_base)
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Dense(1024,activation='relu'))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Dense(512,activation='relu'))
model.add(keras.layers.Dense(3,activation='sigmoid'))

covn_base.trainable = False #设置权重参数不可训练

In [12]:
#编译
model.compile(optimizer=keras.optimizers.Adam(lr=0.001),
             loss = 'sparse_categorical_crossentropy',
             metrics=['acc'])

history = model.fit(
    image_train_ds,
    steps_per_epoch=train_count//BATCH_SIZE,
    epochs=1,
    validation_data=image_val_ds,
    validation_steps=val_count//BATCH_SIZE
)

26/26 [==============================] - 265s 10s/step - loss: 0.9142 - acc: 0.4952 - val_loss: 5.2699 - val_acc: 0.0000e+00


In [13]:
covn_base.trainable=True
len(covn_base.layers)

570

In [15]:
fine_tune_at = -3
for layer in covn_base.layers[:fine_tune_at]:
    layer.trainable = False

model.compile(optimizer=keras.optimizers.Adam(0.0005),
             loss = 'sparse_categorical_crossentropy',
             metrics=['acc'])

initial_epochs=1
fine_tune_epochs=1
total_epoch = initial_epochs+fine_tune_epochs

history = model.fit(
    image_train_ds,
    steps_per_epoch=train_count//BATCH_SIZE,
    epochs=total_epoch,
    initial_epoch=initial_epochs,
    validation_data=image_val_ds,
    validation_steps=val_count//BATCH_SIZE
)

model.save('mstz_model_EfficientNetB52.h5')

Epoch 2/2
26/26 [==============================] - 271s 10s/step - loss: 0.7995 - acc: 0.6238 - val_loss: 4.9840 - val_acc: 0.0000e+00


In [16]:
#加载图片
def load_preprocess_images(path):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image,channels=3)
    image = tf.image.resize(image,[260,260])
    image = tf.cast(image,tf.float32)
    image = image/255
    image = tf.expand_dims(image,0)
    return image

In [26]:
test_image_path = glob.glob('test/*.png')

# test_image_path.sort(key=lambda x:int(x.split('\\')[-1].split('.')[0][1:]))

images = [load_preprocess_images(image_path) for image_path in test_image_path]

In [27]:
len(images)

18

In [28]:
image_count = len(images)
# values = []
# result_dict = {}

for i in range(image_count):
    pred = model.predict(images[i])
    values.append(np.argmax(pred))
    print('.',end='')

..................

In [29]:
#写文件
with open('result.csv','w',encoding='utf-8') as f:
    f.write('image_id,category_id\n')
    [f.write('{0},{1}\n'.format(key.split('\\')[1], value+1)) for (key,value) in zip(test_image_path,values)]